# 🧠 Model B — Language Understanding (Text → Structured Meaning)

**Purpose:** Extract structured, clinically relevant information from patient speech transcripts to support triage and downstream reasoning.

| | |
|---|---|
| **Input** | Cleaned text transcript (output from Model A) |
| **Output** | Fixed-schema JSON + `additional_observations` field |
| **Model** | Google Gemini AI (via `google-generativeai`) |
| **Mode** | Constrained information extraction — no diagnosis, no advice |

### Pipeline Position
```
Model A Output                     Model B Output
(Transcript Text)
       │
       ▼
┌─────────────────────┐       ┌──────────────────────────────┐
│  Transcript +       │──────►│  chief_complaint              │
│  Fixed Schema       │       │  duration                     │
│  (prompt)           │       │  severity                     │
│                     │       │  body_part                    │
│  Gemini Model       │       │  associated_symptoms []       │
│                     │       │  red_flags_present true/false │
└─────────────────────┘       │  additional_observations      │
                              └──────────────────────────────┘
                                         │
                                         ▼
                                    → Model C (Dialogue)
                                    → Doctor Review
```

> ⚠️ **Non-Negotiable Constraints:** This model produces **observational output only**. No diagnoses, no medical advice, no treatment or medication recommendations.

---
## 📦 Section 1 — Install Dependencies

In [ ]:
!pip install -q -U google-generativeai
!pip install -q jsonschema

print("✅ Dependencies installed.")

---
## 🔑 Section 2 — API Key Configuration

Store your Gemini API key securely using **Colab Secrets** (the 🔑 icon in the left sidebar) under the name `GEMINI_API_KEY`. This avoids hardcoding credentials.

In [ ]:
import google.generativeai as genai
from google.colab import userdata

# ── Load API key from Colab Secrets ──────────────────────────────────────────
# Add your key via: left sidebar → 🔑 Secrets → Add "GEMINI_API_KEY"
try:
    GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
    genai.configure(api_key=GEMINI_API_KEY)
    print("✅ Gemini API key loaded from Colab Secrets.")
except Exception:
    # Fallback: paste key directly (not recommended for shared notebooks)
    GEMINI_API_KEY = "YOUR_API_KEY_HERE"  # ← replace only if Secrets unavailable
    genai.configure(api_key=GEMINI_API_KEY)
    print("⚠️  API key set manually. Use Colab Secrets for security.")

---
## 🤖 Section 3 — Model Initialisation

In [ ]:
import json
import re
from dataclasses import dataclass, field, asdict
from typing import List, Optional

# ── Model configuration ───────────────────────────────────────────────────────
# gemini-1.5-flash: fast, cost-efficient, supports multilingual text well
# gemini-1.5-pro  : higher reasoning quality — swap in for production
GEMINI_MODEL_NAME = "gemini-1.5-flash"

generation_config = genai.types.GenerationConfig(
    temperature       = 0.0,    # Deterministic — critical for consistent schema output
    top_p             = 1.0,
    max_output_tokens = 1024,   # Schema output is compact; 1024 is sufficient
)

safety_settings = [
    # Relax Gemini's default blocks on medical content so clinical terms pass through
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT",  "threshold": "BLOCK_ONLY_HIGH"},
    {"category": "HARM_CATEGORY_HARASSMENT",         "threshold": "BLOCK_ONLY_HIGH"},
    {"category": "HARM_CATEGORY_HATE_SPEECH",        "threshold": "BLOCK_ONLY_HIGH"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",  "threshold": "BLOCK_ONLY_HIGH"},
]

gemini_model = genai.GenerativeModel(
    model_name        = GEMINI_MODEL_NAME,
    generation_config = generation_config,
    safety_settings   = safety_settings,
)

print(f"✅ Gemini model initialised: {GEMINI_MODEL_NAME}")
print(f"   Temperature : {generation_config.temperature} (deterministic)")

---
## 📐 Section 4 — Output Schema Definition

The schema is the contract between the model and downstream systems. It is fixed and passed directly into the prompt — Gemini is instructed to populate it, not design it.

In [ ]:
# ── Dataclass for strongly-typed output ───────────────────────────────────────
@dataclass
class ClinicalExtraction:
    """
    Structured output from Model B.
    All fields are observational — no diagnosis, no advice.
    """
    chief_complaint         : str         = ""       # Main reason for visit
    duration                : str         = ""       # How long the symptom has been present
    severity                : str         = ""       # Patient's own description (mild/moderate/severe)
    body_part               : str         = ""       # Anatomical area mentioned
    associated_symptoms     : List[str]   = field(default_factory=list)  # Secondary symptoms
    red_flags_present       : Optional[bool] = None  # True if any red flag language detected
    additional_observations : str         = ""       # Unstructured but clinically relevant context


# ── Empty schema template (injected into prompt) ─────────────────────────────
EMPTY_SCHEMA = {
    "chief_complaint"         : "",
    "duration"                : "",
    "severity"                : "",
    "body_part"               : "",
    "associated_symptoms"     : [],
    "red_flags_present"       : None,
    "additional_observations" : ""
}

# ── Red flag vocabulary ────────────────────────────────────────────────────────
# Used for schema validation — Gemini's red_flags_present is cross-checked
RED_FLAG_TERMS = [
    # English
    "can't breathe", "cannot breathe", "chest pain", "chest tightness",
    "unconscious", "fainted", "fainting", "collapse", "collapsed",
    "severe bleeding", "heavy bleeding", "coughing blood", "blood in urine",
    "stroke", "paralysis", "can't move", "cannot move",
    "seizure", "convulsion", "fits",
    "sudden vision loss", "sudden blindness",
    "difficulty swallowing", "can't swallow",
    # Kinyarwanda transliterations (common terms)
    "guhumeka",        # breathing
    "amaraso",         # blood
    "guhinduka",       # collapse / change suddenly
    "ingufu",          # convulsion / force
    "kunanirwa",       # unable to
    "imitsi",          # paralysis / nerves
]

print("✅ Output schema and red flag vocabulary defined.")
print(f"   Schema fields : {list(EMPTY_SCHEMA.keys())}")
print(f"   Red flag terms: {len(RED_FLAG_TERMS)} terms loaded")

---
## 📝 Section 5 — Prompt Engineering

The system prompt and user prompt are separated. The system prompt establishes role, constraints, and output rules once. The user prompt injects the transcript at runtime.

Design principles applied:
- **Strict JSON-only output** — no prose, no markdown fences
- **Leave blank, never invent** — missing fields stay empty
- **Hard constraint block** — no diagnosis, no advice, no recommendations
- **Multilingual awareness** — Kinyarwanda, English, and mixed text are all valid

In [ ]:
SYSTEM_PROMPT = """\
You are a clinical information extraction assistant operating in a hospital triage system.
Your ONLY job is to extract factual, observable information from patient speech transcripts
and populate a fixed JSON schema.

=== HARD CONSTRAINTS (NEVER VIOLATE) ===
- Do NOT generate any diagnosis, suspected condition, or differential.
- Do NOT provide medical advice, treatment suggestions, or medication recommendations.
- Do NOT infer or guess information not present in the transcript.
- Do NOT add new fields to the schema.
- Do NOT wrap your output in markdown code blocks or backticks.
- Your output must be a single, valid JSON object and nothing else.

=== OUTPUT RULES ===
1. Populate fields ONLY with information explicitly stated or directly implied in the transcript.
2. Leave a field as an empty string "" or empty list [] if information is absent or unclear.
3. "red_flags_present": set to true if the patient mentions any of these — difficulty breathing,
   chest pain, loss of consciousness, heavy bleeding, seizure, sudden paralysis, or inability
   to perform a basic function. Set to false if none apply. Set to null if genuinely unclear.
4. "additional_observations": capture any clinically relevant context that doesn't fit other
   fields (e.g. emotional state, environmental context, patient's own worry, language used).
   Keep it concise and factual. Do not include observations about the recording quality.
5. "severity": use the patient's own words or phrasing. Do not reclassify.
6. The transcript may be in English, Kinyarwanda, or a mix of both. Extract from all languages.

=== SCHEMA TO POPULATE ===
{schema}
"""


def build_user_prompt(transcript: str) -> str:
    """Construct the user-turn prompt for a given transcript."""
    return f"""\
=== PATIENT TRANSCRIPT ===
{transcript.strip()}

=== TASK ===
Extract information from the transcript above and return ONLY the populated JSON schema.
Do not include any text before or after the JSON object.
"""


print("✅ Prompt templates defined.")

---
## ⚙️ Section 6 — Output Parsing & Validation

In [ ]:
def parse_gemini_response(raw_text: str) -> dict:
    """
    Parse the raw Gemini response into a Python dict.
    Handles edge cases: markdown fences, leading/trailing whitespace,
    and truncated responses.
    """
    # Strip markdown fences if Gemini adds them despite instruction
    cleaned = re.sub(r"```(?:json)?\s*", "", raw_text).strip()
    cleaned = re.sub(r"```\s*$", "", cleaned).strip()

    # Extract the first JSON object found
    match = re.search(r"\{.*\}", cleaned, re.DOTALL)
    if not match:
        raise ValueError(f"No JSON object found in model response.\nRaw: {raw_text[:300]}")

    return json.loads(match.group(0))


def validate_and_coerce(raw_dict: dict) -> ClinicalExtraction:
    """
    Validate model output against the schema.
    Coerces types, fills missing keys, and cross-validates red_flags_present
    against the known red flag vocabulary.

    Returns a ClinicalExtraction dataclass instance.
    """
    # ── Ensure all keys are present ───────────────────────────────────────────
    validated = {k: raw_dict.get(k, v) for k, v in EMPTY_SCHEMA.items()}

    # ── Type coercion ─────────────────────────────────────────────────────────
    # String fields
    for str_field in ["chief_complaint", "duration", "severity",
                      "body_part", "additional_observations"]:
        if not isinstance(validated[str_field], str):
            validated[str_field] = str(validated[str_field]) if validated[str_field] else ""

    # List field
    if not isinstance(validated["associated_symptoms"], list):
        val = validated["associated_symptoms"]
        validated["associated_symptoms"] = [val] if val else []

    # Boolean / null field
    rfp = validated["red_flags_present"]
    if rfp not in (True, False, None):
        validated["red_flags_present"] = None

    # ── Cross-validate red_flags_present with vocabulary ──────────────────────
    all_text = " ".join([
        validated["chief_complaint"],
        validated["additional_observations"],
        " ".join(validated["associated_symptoms"]),
    ]).lower()

    keyword_hit = any(term.lower() in all_text for term in RED_FLAG_TERMS)

    # If vocabulary detects red flag but model missed it, override to True
    if keyword_hit and validated["red_flags_present"] is not True:
        validated["red_flags_present"] = True
        print("  ⚠️  Red flag keyword detected — overriding model output to True.")

    return ClinicalExtraction(**validated)


print("✅ Parsing and validation functions defined.")

---
## 🚀 Section 7 — Core Extraction Pipeline

In [ ]:
def extract_clinical_information(
    transcript        : str,
    source_language   : str = "unknown",
    source_confidence : float = 1.0,
    verbose           : bool = True,
) -> dict:
    """
    Full Model B pipeline: transcript → structured clinical extraction.

    Args:
        transcript        : Raw text from Model A (any language).
        source_language   : Language reported by Model A ('kinyarwanda'/'english'/'unknown').
        source_confidence : Confidence score from Model A (0.0–1.0).
        verbose           : Print progress and results.

    Returns:
        dict with keys:
            'extraction'          : ClinicalExtraction dataclass
            'extraction_dict'     : dict version of extraction
            'source_language'     : str
            'source_confidence'   : float
            'raw_model_response'  : str (for debugging)
            'extraction_success'  : bool
            'error'               : str or None
    """
    if verbose:
        print("\n" + "═" * 62)
        print(" 🧠  LANGUAGE UNDERSTANDING PIPELINE — MODEL B")
        print("═" * 62)
        print(f"  Source language   : {source_language}")
        print(f"  Source confidence : {source_confidence:.3f}")
        preview = transcript.strip()[:120]
        print(f"  Transcript preview: {preview}{'...' if len(transcript) > 120 else ''}")

    # ── Guard: refuse to process near-empty transcripts ───────────────────────
    if len(transcript.strip()) < 10:
        if verbose:
            print("  ❌ Transcript too short to extract information.")
        empty = ClinicalExtraction()
        return {
            "extraction"         : empty,
            "extraction_dict"    : asdict(empty),
            "source_language"    : source_language,
            "source_confidence"  : source_confidence,
            "raw_model_response" : "",
            "extraction_success" : False,
            "error"              : "Transcript too short.",
        }

    # ── Build prompt ──────────────────────────────────────────────────────────
    system_with_schema = SYSTEM_PROMPT.format(
        schema=json.dumps(EMPTY_SCHEMA, indent=2)
    )
    user_prompt = build_user_prompt(transcript)

    # ── Call Gemini ───────────────────────────────────────────────────────────
    try:
        if verbose:
            print("\n  🔄 Calling Gemini API...")

        chat = gemini_model.start_chat(history=[
            {"role": "user",  "parts": [system_with_schema]},
            {"role": "model", "parts": ["Understood. I will extract information from transcripts and return only valid JSON matching the provided schema, without diagnosis or medical advice."]},
        ])
        response     = chat.send_message(user_prompt)
        raw_response = response.text

        if verbose:
            print("  ✅ Gemini response received.")

    except Exception as api_err:
        err_msg = str(api_err)
        if verbose:
            print(f"  ❌ Gemini API error: {err_msg}")
        empty = ClinicalExtraction()
        return {
            "extraction"         : empty,
            "extraction_dict"    : asdict(empty),
            "source_language"    : source_language,
            "source_confidence"  : source_confidence,
            "raw_model_response" : "",
            "extraction_success" : False,
            "error"              : err_msg,
        }

    # ── Parse and validate ────────────────────────────────────────────────────
    try:
        raw_dict   = parse_gemini_response(raw_response)
        extraction = validate_and_coerce(raw_dict)
        success    = True
        error      = None
    except Exception as parse_err:
        if verbose:
            print(f"  ⚠️  Parse error: {parse_err}")
        extraction = ClinicalExtraction()
        success    = False
        error      = str(parse_err)

    # ── Display results ───────────────────────────────────────────────────────
    if verbose and success:
        print("\n" + "─" * 62)
        print(" 📋  EXTRACTION RESULT")
        print("─" * 62)
        d = asdict(extraction)
        for key, value in d.items():
            display_val = value if value not in ("", [], None) else "⬜ (not found)"
            print(f"  {key:<28}: {display_val}")
        if extraction.red_flags_present:
            print("\n  🚨 RED FLAG DETECTED — escalate for immediate review.")

    return {
        "extraction"         : extraction,
        "extraction_dict"    : asdict(extraction),
        "source_language"    : source_language,
        "source_confidence"  : source_confidence,
        "raw_model_response" : raw_response,
        "extraction_success" : success,
        "error"              : error,
    }


print("✅ Core extraction pipeline defined.")

---
## 🔗 Section 8 — Model A → Model B Integration

This is the standard runtime flow when Models A and B are chained together.

In [ ]:
def run_pipeline_from_model_a(model_a_output: dict, verbose: bool = True) -> dict:
    """
    Accept the full output dict from Model A (transcribe_audio_file)
    and feed it directly into Model B.

    Args:
        model_a_output : Dict returned by `transcribe_audio_file()` from Model A.
        verbose        : Print pipeline progress.

    Returns:
        Model B output dict (same structure as extract_clinical_information).
    """
    transcript  = model_a_output.get("full_text", "")
    language    = model_a_output.get("dominant_language", "unknown")
    confidence  = model_a_output.get("mean_confidence", 1.0)

    if verbose:
        print("🔗 Receiving Model A output...")
        print(f"   Language   : {language}")
        print(f"   Confidence : {confidence:.3f}")
        print(f"   Transcript : {transcript[:100]}..." if len(transcript) > 100 else f"   Transcript : {transcript}")

    return extract_clinical_information(
        transcript        = transcript,
        source_language   = language,
        source_confidence = confidence,
        verbose           = verbose,
    )


# ─────────────────────────────────────────────────────────────────────────────
# USAGE (when Model A notebook is also open / imported):
#
#   # Model A
#   model_a_result = transcribe_audio_file("patient_audio.wav")
#
#   # Model B — chain directly
#   model_b_result = run_pipeline_from_model_a(model_a_result)
# ─────────────────────────────────────────────────────────────────────────────

print("✅ Model A → Model B integration function defined.")

---
## 🧪 Section 9 — Test Cases

Run these to validate the pipeline before connecting to real audio. Covers English, Kinyarwanda, mixed language, a red flag case, and a sparse/minimal transcript.

In [ ]:
TEST_CASES = [
    {
        "id"         : "TC-01 | English — standard presentation",
        "language"   : "english",
        "confidence" : 0.91,
        "transcript" : """
            I've had a headache for the past three days. It's mostly on the right side of my head.
            The pain is moderate, maybe a six out of ten. I also feel a bit nauseous, and
            light seems to make it worse. I took paracetamol yesterday but it didn't help much.
        """,
    },
    {
        "id"         : "TC-02 | Kinyarwanda — abdominal symptoms",
        "language"   : "kinyarwanda",
        "confidence" : 0.84,
        "transcript" : """
            Ndi n'ububabare mu nda kuva ejo. Ububabare ni uburemere kandi burarushaho. 
            Sinashye neza ijoro rya ejo. Nk'aho mba nshaka kuruka ariko sinabikora.
        """,
        # Translation hint: Stomach pain since yesterday. Heavy pain, getting worse.
        # Couldn't sleep last night. Feels like vomiting but hasn't.
    },
    {
        "id"         : "TC-03 | Mixed language — code-switching",
        "language"   : "kinyarwanda",
        "confidence" : 0.77,
        "transcript" : """
            Chest pain yatangiye ejobundi. Ndi kwibaza niba ni heart problem.
            Ntushobora guhumeka neza iyo ugerageza gukora akazi. 
            The pain shoots to my left arm sometimes.
        """,
        # Translation hint: Chest pain started two days ago. Wondering if it's a heart problem.
        # Can't breathe well when trying to work. Pain goes to left arm.
    },
    {
        "id"         : "TC-04 | English — sparse transcript",
        "language"   : "english",
        "confidence" : 0.60,
        "transcript" : "My leg hurts.",
    },
    {
        "id"         : "TC-05 | English — red flag case",
        "language"   : "english",
        "confidence" : 0.95,
        "transcript" : """
            I suddenly can't see properly out of my left eye. It started about an hour ago.
            I also feel very confused and my right arm feels weak. I've never had this before.
        """,
    },
]

print(f"✅ {len(TEST_CASES)} test cases loaded. Run the next cell to execute them.")

In [ ]:
# ── Run all test cases ────────────────────────────────────────────────────────
test_results = {}

for tc in TEST_CASES:
    print(f"\n{'▓' * 62}")
    print(f"  TEST CASE: {tc['id']}")
    print(f"{'▓' * 62}")

    result = extract_clinical_information(
        transcript        = tc["transcript"],
        source_language   = tc["language"],
        source_confidence = tc["confidence"],
        verbose           = True,
    )
    test_results[tc["id"]] = result

print(f"\n\n✅ All {len(TEST_CASES)} test cases completed.")

---
## 🗂️ Section 10 — Batch Processing & Export

In [ ]:
import datetime

def batch_extract(
    transcripts : list,
    verbose     : bool = False,
) -> list:
    """
    Batch process a list of transcript dicts.

    Each item in `transcripts` should have:
        {
          'transcript'       : str,
          'source_language'  : str,   (optional)
          'source_confidence': float  (optional)
        }

    Returns:
        List of Model B output dicts.
    """
    results = []
    total   = len(transcripts)
    for i, item in enumerate(transcripts):
        print(f"  [{i+1}/{total}] Processing transcript...")
        res = extract_clinical_information(
            transcript        = item.get("transcript", ""),
            source_language   = item.get("source_language", "unknown"),
            source_confidence = item.get("source_confidence", 1.0),
            verbose           = verbose,
        )
        res["input_transcript"] = item.get("transcript", "")
        results.append(res)

        # Show one-line summary per item
        ext = res["extraction"]
        flag = "🚨" if ext.red_flags_present else "✅"
        print(f"     {flag} complaint='{ext.chief_complaint or 'N/A'}' "
              f"| severity='{ext.severity or 'N/A'}' "
              f"| confidence={res['source_confidence']:.2f}")

    return results


def export_batch_results(results: list, filepath: str = "model_b_extractions.json"):
    """Save batch results to a JSON file and download it."""
    export_data = {
        "timestamp"  : datetime.datetime.now().isoformat(),
        "model"      : GEMINI_MODEL_NAME,
        "total"      : len(results),
        "red_flags"  : sum(1 for r in results if r["extraction"].red_flags_present),
        "extractions": [
            {
                "index"             : i + 1,
                "source_language"   : r["source_language"],
                "source_confidence" : r["source_confidence"],
                "extraction_success": r["extraction_success"],
                "error"             : r.get("error"),
                "extraction"        : r["extraction_dict"],
            }
            for i, r in enumerate(results)
        ]
    }

    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(export_data, f, ensure_ascii=False, indent=2)

    print(f"\n✅ Saved {len(results)} extraction(s) to: {filepath}")
    print(f"   Red flags detected: {export_data['red_flags']}")

    from google.colab import files
    files.download(filepath)


print("✅ Batch processing and export functions defined.")
print()
print("USAGE EXAMPLE:")
print("  transcripts = [")
print("    {'transcript': 'I have a headache...', 'source_language': 'english'},")
print("    {'transcript': 'Ndi n\'ububabare...', 'source_language': 'kinyarwanda'},")
print("  ]")
print("  results = batch_extract(transcripts)")
print("  export_batch_results(results)")

---
## 📊 Section 11 — Prompt Calibration & Validation Workflow

Use this section to calibrate prompts against your manually annotated local dataset. Feed in annotated examples and compare model output against gold labels.

In [ ]:
def evaluate_against_annotations(
    annotated_examples: list,
    fields_to_evaluate: list = ["chief_complaint", "severity", "body_part"],
) -> dict:
    """
    Compare model extractions against manually annotated gold labels.

    Args:
        annotated_examples : List of dicts, each with:
            {
              'transcript'   : str,
              'gold_labels'  : { 'chief_complaint': ..., 'severity': ..., ... }
            }
        fields_to_evaluate : Which fields to score.

    Returns:
        Dict with per-field match rates and full comparison table.
    """
    field_matches  = {f: 0 for f in fields_to_evaluate}
    field_totals   = {f: 0 for f in fields_to_evaluate}
    comparisons    = []

    for i, ex in enumerate(annotated_examples):
        print(f"\n  [{i+1}/{len(annotated_examples)}] Evaluating...")
        result   = extract_clinical_information(ex["transcript"], verbose=False)
        pred     = result["extraction_dict"]
        gold     = ex["gold_labels"]
        row      = {"index": i+1, "transcript_preview": ex["transcript"][:80]}

        for fld in fields_to_evaluate:
            gold_val = str(gold.get(fld, "")).strip().lower()
            pred_val = str(pred.get(fld, "")).strip().lower()
            # Soft match: gold label contained in prediction or exact match
            match    = (gold_val in pred_val) or (gold_val == pred_val)
            field_matches[fld] += int(match)
            field_totals[fld]  += 1
            row[f"{fld}_gold"]  = gold_val
            row[f"{fld}_pred"]  = pred_val
            row[f"{fld}_match"] = "✅" if match else "❌"
            print(f"     {fld}: gold='{gold_val}' | pred='{pred_val}' | {'✅' if match else '❌'}")

        comparisons.append(row)

    # Summary
    print("\n" + "─" * 50)
    print(" CALIBRATION SUMMARY")
    print("─" * 50)
    summary = {}
    for fld in fields_to_evaluate:
        rate = field_matches[fld] / max(field_totals[fld], 1)
        summary[fld] = round(rate, 3)
        bar = "█" * int(rate * 20) + "░" * (20 - int(rate * 20))
        print(f"  {fld:<28}: {bar}  {rate*100:.1f}%")

    return {"per_field_accuracy": summary, "comparisons": comparisons}


# ── Example annotated set — replace with your real annotations ────────────────
SAMPLE_ANNOTATED = [
    {
        "transcript": "I've had a headache for two days, mostly on the right side. It's quite severe.",
        "gold_labels": {"chief_complaint": "headache", "duration": "two days", "severity": "severe", "body_part": "head"},
    },
    {
        "transcript": "My stomach has been hurting since this morning. Moderate pain.",
        "gold_labels": {"chief_complaint": "stomach pain", "duration": "since this morning", "severity": "moderate", "body_part": "stomach"},
    },
]

# Uncomment to run calibration:
# calibration_results = evaluate_against_annotations(SAMPLE_ANNOTATED)

print("✅ Calibration framework defined.")
print("   Uncomment the last line to run with SAMPLE_ANNOTATED or your own dataset.")

---
## 📝 Notes & Design Decisions

| Topic | Detail |
|---|---|
| **Temperature = 0.0** | Ensures deterministic, reproducible schema outputs — essential for clinical consistency. |
| **Red flag override** | A keyword vocabulary layer cross-checks Gemini's `red_flags_present` to catch any model misses. |
| **No diagnosis constraint** | Enforced both in the system prompt and in downstream validation. The model sees no diagnosis instruction at all. |
| **`additional_observations`** | Intentionally flexible — preserves patient affect, contextual clues, and language-switch patterns that don't map to schema fields. |
| **Kinyarwanda support** | Gemini 1.5 has multilingual support. Kinyarwanda extraction quality should be validated against your annotated set and the prompt refined if needed. |
| **Low-confidence transcripts** | If `source_confidence < 0.5`, consider flagging the extraction result as provisional for doctor review. |
| **Model swap** | Replace `gemini-1.5-flash` with `gemini-1.5-pro` in `GEMINI_MODEL_NAME` for higher extraction quality in production. |